In [1]:
import tensorflow as tf
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import cv2 
import numpy as np

In [2]:
# Set the input image size
input_size = (512, 512)

In [3]:

# Load the Inception v3 model without the top classification layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))


In [53]:
# Freeze the pre-trained layers
for layer in base_model.layers[:50]:
    layer.trainable = False


In [5]:
# Add a new classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # 4 output units for the tumor classes


In [6]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
def pre_proc_fn(img):    
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(src=image,ksize=(5,5),sigmaX=0,sigmaY=0)
    ##image = cv2.equalizeHist(src=image, )
    image = np.expand_dims(image, axis=-1)
    return image

In [33]:
def preproc_fn(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [8]:

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function=pre_proc_fn
)

In [35]:
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    preprocessing_function= preproc_fn,
)

In [10]:
 
path = 'C:\\Users\\Abhijith_Kumble\\Desktop\\IEEE\\brain_Tumor_final\\brain_tumor'

# Set the paths to your training and testing data
train_data_dir = path+'\\Training'
test_data_dir = path + '\\Testing'




In [11]:
image_size = (112,112)
# Set the batch size
batch_size = 32

# Generate training and testing data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'rgb'
    
)


Found 5712 images belonging to 4 classes.


In [34]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'rgb',
    shuffle=False,

    
)


Found 1311 images belonging to 4 classes.


In [13]:

# Set the number of training and testing samples
num_train_samples = train_generator.samples
num_test_samples = test_generator.samples

# Set the number of training epochs
num_epochs = 10

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 256, 256, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 256, 256, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 256, 256, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 256, 256, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 256, 256, 32)     128       
 ation)                                                      

In [15]:
print(len(model.layers))

89


In [54]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=num_epochs,

)

Epoch 1/10
178/178 [==============================] - 474s 3s/step - loss: 0.1487 - accuracy: 0.9472
Epoch 2/10
178/178 [==============================] - 461s 3s/step - loss: 0.1462 - accuracy: 0.9458
Epoch 3/10
178/178 [==============================] - 456s 3s/step - loss: 0.1344 - accuracy: 0.9525
Epoch 4/10
178/178 [==============================] - 458s 3s/step - loss: 0.1316 - accuracy: 0.9509
Epoch 5/10
178/178 [==============================] - 456s 3s/step - loss: 0.1317 - accuracy: 0.9491
Epoch 6/10
178/178 [==============================] - 459s 3s/step - loss: 0.1251 - accuracy: 0.9542
Epoch 7/10
178/178 [==============================] - 452s 3s/step - loss: 0.1191 - accuracy: 0.9576
Epoch 8/10
178/178 [==============================] - 452s 3s/step - loss: 0.0970 - accuracy: 0.9629
Epoch 9/10
178/178 [==============================] - 422s 2s/step - loss: 0.1153 - accuracy: 0.9588
Epoch 10/10
178/178 [==============================] - 420s 2s/step - loss: 0.1034 - accura

In [55]:
model.save('mobilenet_ver2.0.h5')

In [56]:
model = tf.keras.models.load_model('mobilenet_ver2.0.h5')

In [57]:
predictions = model.predict(test_generator)

41/41 [==============================] - 69s 2s/step


In [37]:
print(predictions)

[[9.9984956e-01 1.3632620e-04 1.3839308e-05 2.8350550e-07]
 [9.5211649e-01 2.4907785e-02 4.5720395e-03 1.8403685e-02]
 [9.9991572e-01 6.9866102e-05 1.4287366e-05 1.3927546e-07]
 ...
 [1.0180511e-06 6.9609625e-05 3.4975035e-06 9.9992585e-01]
 [4.3549790e-06 3.6185794e-04 1.0582827e-05 9.9962318e-01]
 [2.6487101e-07 7.2749041e-05 1.4491902e-06 9.9992561e-01]]


In [58]:
np.save('predictions_mobilenet.npy',predictions)

In [59]:
predicted_labels = predictions.argmax(axis=1)

In [60]:
from sklearn.metrics import accuracy_score,recall_score,f1_score,confusion_matrix

In [61]:
true_labels = np.array(test_generator.labels)

In [62]:
accuracy = accuracy_score(true_labels,predicted_labels, )

In [63]:
f1 = f1_score(true_labels,predicted_labels,average='weighted')

In [64]:
recall = recall_score(true_labels,predicted_labels,average='weighted')

In [65]:
print('accuracy  =' , round(accuracy*100,4))
print('f1        =' , round(f1,6))
print('recall    =' , round(recall,6))


accuracy  = 89.0923
f1        = 0.885113
recall    = 0.890923


In [66]:
confusion_matrix(true_labels,predicted_labels)

array([[284,   7,   0,   9],
       [ 18, 183,  14,  91],
       [  0,   3, 401,   1],
       [  0,   0,   0, 300]], dtype=int64)